DSI BOS 11 (May 2020) Project 5

Alex Golden, Jungmoon Ham, Luke Podsiadlo, Zach Tretter

Workbook 6 - Mapping

----------

## Geocode and Map Addresses from Police Radio Speech-to-Text

Relevance to Problem Statement : _"The tool will flag neighborhoods or specific streets where the police and first-respondents were called to provide assistance related to the event."_

#### Workflow Steps

1. Import dataframe.  Addresses at minimum data, additional context as avaiable

2. Find Latitude/Longitude from address ("geocode") via google to enable plotting on map

3. Visualize on map using [Folium](https://python-visualization.github.io/folium/)

#### API Key (Google Geocoding API)

In [ ]:
# Text String from Google
my_key = "<INSERT KEY HERE>"

_____________

## Imports

In [ ]:
import pandas as pd
import numpy as np
import requests

# !pip install folium
import folium

### Import Dataframe

In [ ]:
## Import

In [ ]:
# Addresses to test folium
test_addresses = ["MIT","Boston College","Harvard","Tufts",'Boston University']

In [ ]:
# Identify addresses to plot
addresses_to_plot = test_addresses

### Function that Finds Latitude and Longitude from Address

Function code adapted from [article on geocoding written by Shane Lynn](https://www.shanelynn.ie/batch-geocoding-in-python-with-google-geocoding-api/)

Given an address, this code will create a URL to query google.

It will return this dictionary

| Key                 | Value             | Information                                              |
|---------------------|-------------------|----------------------------------------------------------|
| lat_long            | tuple (lat, long) | Geocode result from google                               |
| full_output_address | Text String       | Full address corresponding to geocode returned by google |
| input_string        | Text String       | String that was searched on (from dataframe)             |
| number_of_results   | Integer           | Number of results                                        |
| status              | Text STring       | 'Ok' if request successful<br>None otherwise             |

In [ ]:
def get_google_latlong(address,
                       api_key=None,
                       return_full_response=False):
    
    # Create the URL for geocoding
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    
    # Add the API key
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the results:
    results = requests.get(geocode_url)
    
    # Display results as JSON dictionary
    results = results.json()

    # If no result, return None
    if len(results['results']) == 0:
        output = {
            "latitude": None,
            "longitude": None
        }
        
    else:    
        answer = results['results'][0]
        output = { "lat_long" : (
            answer.get('geometry').get('location').get('lat'),
            answer.get('geometry').get('location').get('lng')
        )
                 }
        
        output['full_output_address'] = answer.get('formatted_address')
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')

    if return_full_response is True:
        output['response'] = results
    
    return output

#### Extract just the latitude and longitude

In [ ]:
def extract_lat_long(list_of_addresses):
        return [get_google_latlong(i,
                                   api_key = my_key)['lat_long'] 
                for i in list_of_addresses]

### Folium Map

In [ ]:
# Create a Folium Object (Centered at Harvard Square)
m = folium.Map(
    location = [42.3736, -71.1190],
    tiles = 'stamentoner',
    zoom_start = 12,
    control_scale=False
)

# Iterate though our list of latitude and longitudes
for i in extract_lat_long(addresses_to_plot):
    folium.CircleMarker(
    radius = 10,
    location = i,
    color ='red'
    ).add_to(m)

# Display the map
m